# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 2:08PM,249548,10,Enova-10893,Emerginnova,Released
1,Oct 27 2022 2:07PM,249547,19,ADV80007140,"AdvaGen Pharma, Ltd",Released
2,Oct 27 2022 2:07PM,249547,19,ADV80007141,"AdvaGen Pharma, Ltd",Released
3,Oct 27 2022 2:07PM,249547,19,ADV80007142,"AdvaGen Pharma, Ltd",Released
4,Oct 27 2022 2:07PM,249547,19,ADV80007143,"AdvaGen Pharma, Ltd",Released
5,Oct 27 2022 2:07PM,249547,19,ADV80007144,"AdvaGen Pharma, Ltd",Released
6,Oct 27 2022 2:04PM,249546,10,CTF0011477,"Citieffe, Inc.",Released
7,Oct 27 2022 2:04PM,249546,10,CTF0011478,"Citieffe, Inc.",Released
8,Oct 27 2022 1:59PM,249545,10,1144,"Amcyte Pharma, Inc.",Released
9,Oct 27 2022 1:48PM,249544,15,PNC391358A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,249544,Released,36
26,249545,Released,1
27,249546,Released,2
28,249547,Released,5
29,249548,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249544,NaN,NaN,36.0
249545,NaN,NaN,1.0
249546,NaN,NaN,2.0
249547,NaN,NaN,5.0
249548,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248349,0.0,1.0,0.0
249472,25.0,15.0,1.0
249479,0.0,1.0,0.0
249485,0.0,1.0,0.0
249494,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248349,0,1,0
249472,25,15,1
249479,0,1,0
249485,0,1,0
249494,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248349,0,1,0
1,249472,25,15,1
2,249479,0,1,0
3,249485,0,1,0
4,249494,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248349,,1,
1,249472,25,15,1
2,249479,,1,
3,249485,,1,
4,249494,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 2:08PM,249548,10,Emerginnova
1,Oct 27 2022 2:07PM,249547,19,"AdvaGen Pharma, Ltd"
6,Oct 27 2022 2:04PM,249546,10,"Citieffe, Inc."
8,Oct 27 2022 1:59PM,249545,10,"Amcyte Pharma, Inc."
9,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc."
45,Oct 27 2022 1:40PM,249543,16,"SHL Pharma, LLC"
46,Oct 27 2022 1:23PM,249541,10,ISDIN Corporation
54,Oct 27 2022 1:23PM,249540,10,ISDIN Corporation
94,Oct 27 2022 1:14PM,249537,19,"AdvaGen Pharma, Ltd"
97,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 27 2022 2:08PM,249548,10,Emerginnova,,,1
1,Oct 27 2022 2:07PM,249547,19,"AdvaGen Pharma, Ltd",,,5
2,Oct 27 2022 2:04PM,249546,10,"Citieffe, Inc.",,,2
3,Oct 27 2022 1:59PM,249545,10,"Amcyte Pharma, Inc.",,,1
4,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc.",,,36
5,Oct 27 2022 1:40PM,249543,16,"SHL Pharma, LLC",,,1
6,Oct 27 2022 1:23PM,249541,10,ISDIN Corporation,,8,
7,Oct 27 2022 1:23PM,249540,10,ISDIN Corporation,,37,3
8,Oct 27 2022 1:14PM,249537,19,"AdvaGen Pharma, Ltd",,,3
9,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",,,39


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 2:08PM,249548,10,Emerginnova,1,,
1,Oct 27 2022 2:07PM,249547,19,"AdvaGen Pharma, Ltd",5,,
2,Oct 27 2022 2:04PM,249546,10,"Citieffe, Inc.",2,,
3,Oct 27 2022 1:59PM,249545,10,"Amcyte Pharma, Inc.",1,,
4,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc.",36,,
5,Oct 27 2022 1:40PM,249543,16,"SHL Pharma, LLC",1,,
6,Oct 27 2022 1:23PM,249541,10,ISDIN Corporation,,8,
7,Oct 27 2022 1:23PM,249540,10,ISDIN Corporation,3,37,
8,Oct 27 2022 1:14PM,249537,19,"AdvaGen Pharma, Ltd",3,,
9,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",39,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 2:08PM,249548,10,Emerginnova,1,,
1,Oct 27 2022 2:07PM,249547,19,"AdvaGen Pharma, Ltd",5,,
2,Oct 27 2022 2:04PM,249546,10,"Citieffe, Inc.",2,,
3,Oct 27 2022 1:59PM,249545,10,"Amcyte Pharma, Inc.",1,,
4,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc.",36,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 2:08PM,249548,10,Emerginnova,1.0,NaN,NaN
1,Oct 27 2022 2:07PM,249547,19,"AdvaGen Pharma, Ltd",5.0,NaN,NaN
2,Oct 27 2022 2:04PM,249546,10,"Citieffe, Inc.",2.0,NaN,NaN
3,Oct 27 2022 1:59PM,249545,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN
4,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc.",36.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 2:08PM,249548,10,Emerginnova,1.0,0.0,0.0
1,Oct 27 2022 2:07PM,249547,19,"AdvaGen Pharma, Ltd",5.0,0.0,0.0
2,Oct 27 2022 2:04PM,249546,10,"Citieffe, Inc.",2.0,0.0,0.0
3,Oct 27 2022 1:59PM,249545,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0
4,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc.",36.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1497237,8.0,45.0,0.0
15,1746598,108.0,16.0,26.0
16,1247582,4.0,1.0,0.0
19,499084,8.0,0.0,0.0
20,498979,0.0,2.0,0.0
21,996922,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1497237,8.0,45.0,0.0
1,15,1746598,108.0,16.0,26.0
2,16,1247582,4.0,1.0,0.0
3,19,499084,8.0,0.0,0.0
4,20,498979,0.0,2.0,0.0
5,21,996922,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,8.0,45.0,0.0
1,15,108.0,16.0,26.0
2,16,4.0,1.0,0.0
3,19,8.0,0.0,0.0
4,20,0.0,2.0,0.0
5,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,8.0
1,15,Released,108.0
2,16,Released,4.0
3,19,Released,8.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,26.0,0.0,0.0,0.0,0.0
Executing,45.0,16.0,1.0,0.0,2.0,1.0
Released,8.0,108.0,4.0,8.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,26.0,0.0,0.0,0.0,0.0
1,Executing,45.0,16.0,1.0,0.0,2.0,1.0
2,Released,8.0,108.0,4.0,8.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,26.0,0.0,0.0,0.0,0.0
1,Executing,45.0,16.0,1.0,0.0,2.0,1.0
2,Released,8.0,108.0,4.0,8.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()